# RNA分析
- mkref：构建参考基因组
- sample：生成样本信息
- barcode：解复用barcode，主要过滤fastq文件中不包含barcode的reads和低质量的reads
- cutadapt：对reads进行trimming
- star：与参考基因组进行比对
- featureCount：转录组定量分析
- count：区分barcode是否为背景，生成表达矩阵
- analysis：降维，聚类的可视化，差异基因表

1.构建参考基因组

mkdir hs_ensembl_99

cd hs_ensembl_99

wget ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

wget ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz

gunzip Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

gunzip Homo_sapiens.GRCh38.99.gtf.gz

conda activate celescope

celescope rna mkref \

 --genome_name Homo_sapiens_ensembl_99 \

 --fasta Homo_sapiens.GRCh38.dna.primary_assembly.fa \

 --gtf Homo_sapiens.GRCh38.99.gtf

2.创建mapfile文件，multi_rna命令生成shell文件
```
multi_rna\
  --mapfile rna.mapfile\
  --genomeDir /SGRNJ/Public/Database/genome/homo_mus\
  --thread 8\
  --mod shell
```
- cat rna.mapfile\
rna1&emsp;../../celescope_test_data/rna1/fastqs&emsp;test1

3.运行生成的shell文件

sh shell/test1.sh
```
celescope rna sample --outdir .//test1/00.sample --sample test1 --thread 4 --chemistry auto  --fq1 ../../celescope_test_data/rna/fastqs/rna_1.fq.gz

celescope rna barcode --outdir .//test1/01.barcode --sample test1 --thread 4 --chemistry auto --lowNum 2  --fq1 ../../celescope_test_data/rna/fastqs/rna_1.fq.gz --fq2 ../../celescope_test_data/rna/fastqs/rna_2.fq.gz
celescope rna cutadapt --outdir .//test1/02.cutadapt --sample test1 --thread 4 --minimum_length 20 --nextseq_trim 20 --overlap 10 --insert 150  --fq .//test1/01.barcode/test1_2.fq

celescope rna star --outdir .//test1/03.star --sample test1 --thread 4 --genomeDir /SGRNJ03/randd/public/genome/rna/hs_ensembl_99 --outFilterMultimapNmax 1 --starMem 30  --fq .//test1/02.cutadapt/test1_clean_2.fq

celescope rna featureCounts --outdir .//test1/04.featureCounts --sample test1 --thread 4 --gtf_type exon --genomeDir /SGRNJ03/randd/public/genome/rna/hs_ensembl_99  --input .//test1/03.star/test1_Aligned.sortedByCoord.out.bam

celescope rna count --outdir .//test1/05.count --sample test1 --thread 4 --genomeDir /SGRNJ03/randd/public/genome/rna/hs_ensembl_99 --expected_cell_num 3000 --cell_calling_method auto  --bam .//test1/04.featureCounts/test1_name_sorted.bam --force_cell_num None

celescope rna analysis --outdir .//test1/06.analysis --sample test1 --thread 4 --genomeDir /SGRNJ03/randd/public/genome/rna/hs_ensembl_99  --matrix_file .//test1/05.count/test1_filtered_feature_bc_matrix

```

# reference
- test_script:

https://github.com/singleron-RD/celescope_test_script

- test_data:

https://github.com/singleron-RD/celescope_test_data


